In [ ]:
import cv2
import fitz
import os
import numpy as np
from google.colab.patches import cv2_imshow
from docx import Document
from docx.shared import Inches
import spacy
import json
import PyPDF2


file = 'ati12052021_8.pdf'
pdf = fitz.open(file)
dir_path = "/content/ITStudio/Images"
for page_number in range(len(pdf)):
    image_list = pdf.get_page_images(page_number)
    for image in image_list:
        xref = image[0]
        base_image = pdf.extract_image(xref)
        image_data = base_image["image"]

        # Save the image data to a file
        with open(f"{dir_path}/image{xref}_page{page_number}.png", "wb") as img_file:
            img_file.write(image_data)

doc = Document()

class EntityGenerator(object):
    def __init__(self, text=None):
        self.text = text

    def get(self):
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(self.text)
        text = [ent.text for ent in doc.ents]
        entity = [ent.label_ for ent in doc.ents]

        from collections import Counter
        data = Counter(zip(entity))
        unique_entity = list(data.keys())
        unique_entity = [x[0] for x in unique_entity]

        d = {}
        for val in unique_entity:
            d[val] = []

        for key,val in dict(zip(text, entity)).items():
            if val in unique_entity:
                d[val].append(key)
        return d

class Resume(object):
    def __init__(self, filename=None):
        self.filename = filename

    def get(self):
        fFileObj = open(self.filename, 'rb')
        pdfReader = PyPDF2.PdfReader(fFileObj)
        print("Total Pages : {} ".format(len(pdfReader.pages)))

        entities = {}
        for i in range(len(pdfReader.pages)):
            pageObj = pdfReader.pages[i]
            resume = pageObj.extract_text()
            helper = EntityGenerator(text=resume)
            entities[f'Page {i+1}'] = helper.get()
        return entities

resume = Resume(filename="ati12052021_8.pdf")
entities = resume.get()

doc.add_heading('ENTITIES:', level=1)
for page, entity in entities.items():
    doc.add_heading(page, level=1)
    for key, values in entity.items():
        doc.add_heading(key, level=2)
        for value in values:
            doc.add_paragraph(value)

# Save entities to a text file
with open('entities.txt', 'w') as f:
    f.write(json.dumps(entities, indent=3))


# Load YOLOv4
net = cv2.dnn.readNet('/content/ITStudio/yolov4.weights', '/content/ITStudio/yolov4.cfg')
layer_names = net.getUnconnectedOutLayersNames()
with open('/content/ITStudio/coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Function for object detection using YOLOv4
def perform_object_detection(image_path, i):
    image = cv2.imread(image_path)
    if image is None:
      print("Image not loaded correctly")
      return
    height, width, _ = image.shape

    # Convert image to blob format
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Forward pass and get bounding box predictions
    detections = net.forward(layer_names)

    # Process detections
    boxes = []
    confidences = []
    class_ids = []

    for output in detections:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # Adjust confidence threshold as needed
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype(int)
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to remove overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Extract labeled objects
    labeled_objects = [(class_ids[i], confidences[i], boxes[i]) for i in indices]

    # Draw bounding boxes, class names, and confidence scores
    for object in labeled_objects:
        class_id, confidence, box = object
        x, y, w, h = box
        label = str(class_id)
        class_name = classes[class_id]
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, class_name + " " + str(round(confidence, 2)), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display image
    cv2_imshow(image)

    
    cv2.imwrite(f'/content/ITStudio/Detected Images/result{i}.png', image)
    
# Get list of all image files in the directory
image_dir = '/content/ITStudio/Images'
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.png') or f.endswith('.jpg')]

# Perform object detection on each image
for i, image_file in enumerate(image_files):
    perform_object_detection(image_file, i)
directory = '/content/ITStudio/Detected Images'

# Use os.listdir to get a list of filenames in the directory
doc.add_heading('OBJECT DETECTIONS:', level=1)
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    if not os.path.isfile(filepath):
        continue  # Skip to the next
    doc.add_picture(filepath)

# Save the Word document
doc.save('/content/ITStudio/Detected_Images.docx')
